In [1]:
%reload_ext autoreload

In [2]:
%autoreload 2
import math
import numpy as np

from bisqt import BoseHubbardHamiltonian

from qiskit.quantum_info import Pauli, Operator
from qiskit.quantum_info import SparsePauliOp

from symmer.operators import PauliwordOp, QuantumState
from symmer.projection import QubitTapering 

import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)

from symmer.operators import IndependentOp
from scipy.linalg import eigh, eig
import scipy.sparse

C:\Users\User\OneDrive\Desktop\Martina\Tesi\tapenv\lib\site-packages\cotengra\hyperoptimizers\hyper.py:34: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [3]:
#line lattice 
num_nodes = 3 #number of sites in the lattice 
boundary_condition = BoundaryCondition.OPEN #open lattice
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

In [4]:
#Hamiltonian of the system given the number of sites in the lattice and the number of qubits
num_qubits = 2
num_nodes = 3
H_instance = BoseHubbardHamiltonian(num_nodes, num_qubits, line_lattice)

energy_values =[0.8,1.]

H_q = H_instance.get_H(energy_values[0],energy_values[1])
H = PauliwordOp.from_qiskit(H_q)
#print(H)

AssertionError: Must supply a PauliSumOp

In [ ]:
#TAPERING

#IndipendentOp represents algebraically independent sets of Pauli operators for stabilizer manipulation/projections
IndependentOp.symmetry_generators(H)

In [ ]:
taper_hamiltonian = QubitTapering(H, 'X')
taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis()


In [ ]:
#TAPERED HAMILTONIAN +

sec_array = [1]
#sec_array = [1,1,1,1,1,1]
ham_tap_p= taper_hamiltonian.taper_it(sector=sec_array)




In [ ]:
#TAPERED HAMILTONIAN -

sec_array = [-1]
#sec_array = [-1,-1,-1,-1,-1,-1]
ham_tap_n= taper_hamiltonian.taper_it(sector=sec_array)
